#Запуск алгоритма

In [1]:
#@title Установка пакетов
!pip  install langchain==0.0.335 openai==1.2.3 tiktoken==0.5.1 pydantic==1.10.8 faiss-cpu==1.7.4 nltk oauth2client >/dev/null
!pip install gspread

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.


In [9]:
#@title Импорт библиотек

import os
import getpass
import requests

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import openai
from openai import OpenAI
import tiktoken
import re
import requests
from langchain.docstore.document import Document

import pandas as pd
from google.oauth2.service_account import Credentials

from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json
import numpy as np


# для memory
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI as lc_OpenAI

from datetime import datetime

In [3]:
#@title Опредение функций

def load_text_from_github(kdb_link):
## правильная ссылка: https://github.com/terrainternship/GPT_LaserLove/raw/main/instruction_1.txt
## неправильная ссылка:  https://github.com/terrainternship/GPT_LaserLove/blob/main/instruction_1.txt
  response = requests.get(kdb_link)
  txt = response.text
  return txt

def load_googledoc_by_url(doc_url) -> str: # Функция load_googledoc_by_url предназначена для загрузки текста из гуглдока, по ссылке (doc_url)
  # Extract the document ID from the URL
  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', doc_url)
  if match_ is None:
    raise ValueError('Invalid Google Docs URL')
  doc_id = match_.group(1)

  # Download the document as plain text
  response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
  response.raise_for_status()
  return response.text

def load_document_text(file_path) -> str:   # Функция load_document_text предназначена для загрузки текста из файла, расположенного по указанному пути (file_path)
#    with open(file_path, 'r', encoding='windows-1251') as file:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text_encoded = text.encode('utf-8')
    text = text_encoded.decode('utf-8')
    return text

def load_text(any_link):
  if len(any_link)==0:
    text=''
  else:
    if "github.com" in any_link:
      if "blob" in any_link: any_link=any_link.replace("blob", "raw")
      text = load_text_from_github(any_link)
    elif "docs.google.com" in any_link:
      text = load_googledoc_by_url(any_link)
    else:
      text = load_document_text(any_link)
    return text


def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

def split_text(text, max_count, chunk_overlap, verbose=0, double_split=1):
    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
                          ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:

        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = num_tokens(fragment.page_content)
            print(f"Tokens in text fragment = {count}\n{'-' * 5}\n{fragment.page_content}\n{'=' * 20}")
        if double_split:
          # Разбиение фрагмента текста на части заданной длины с помощью разделителя
          # и добавление каждой части в список source_chunks  и передача в чанк метадата из маркдауновскго сплиттера
          source_chunks.extend(Document(page_content=chunk, metadata=fragment.metadata) for chunk in splitter.split_text(fragment.page_content))
        else:
          source_chunks = fragments

    # Возвращение списка фрагментов текста
    return source_chunks

def create_search_index(data, chunk_size, chunk_overlap, verbo, double_split):
    source_chunks = []
    source_chunks = split_text(text=data, max_count=chunk_size, chunk_overlap=chunk_overlap, verbose=verbo, double_split=double_split)
    return FAISS.from_documents(source_chunks, OpenAIEmbeddings())

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start> assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def answer_index(system, topic, instructions, search_index, temp, verbose, k, model, hist=''):
    docs = search_index.similarity_search_with_score(topic, k=k)
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nТексты для анализа:\n{message_content}\n\nИстория чата:\n{hist}\n\nВопрос клиента: {topic}"}]

    completion = openai.chat.completions.create(model=model, messages=messages, temperature=temp)
    return message_content, completion.choices[0].message.content

#Получение ключа API

In [8]:
#@title Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Тест моделей

In [11]:
#@title 1. Тест по вопросам к БЗ - вопрос задаем вручную с поддержкой памяти
model = "gpt-3.5-turbo-1106"
knowledge_base_link = "https://github.com/terrainternship/GPT_LaserLove/blob/main/knowledge.md?raw=true"
temperature = 0.4
num_fragment = 5
verbose = 0
#system_prompt_link= "https://github.com/terrainternship/GPT_LaserLove/raw/main/%D0%9F%D0%A0%D0%9E%D0%9C%D0%A2%20LaserLove%20Svetl.txt"
#instructions_link = "https://docs.google.com/document/d/18BARvMyB0ZZ0LhAwdxyb_PSeJ0k9Rhg1hZuKr2dLeoc"

#system_prompt = load_text(system_prompt_link)
system_prompt = '''
As a neuro-consultant and neuro-sales manager at LaserLove, please respond to the client's question without using greeting phrases.
If the client has provided their name, address them by name. Focus on providing a direct, clear, and informative answer.
In your messages, concentrate directly on the questions and answers, actively promoting laser hair removal, LPG massage, hardware cosmetology,
and LaserLove cosmetics. Make your responses vivid and attractive, using emojis, lists, and positive expressions to maintain client interest.
Conclude each response with a question that nudges the client towards the next step: booking a procedure or making a purchase.
When the client is ready to buy a subscription, highlight the advantages of long-term use of services and purchasing a subscription.
In case of client's doubts, engage in a dialogue, clarifying and discussing their concerns, offering thoughtful answers.
During casual conversations, demonstrate deep knowledge about the services, share interesting facts, and maintain a friendly tone.
Always respond to client questions in Russian'''

#instructions = load_text(instructions_link)
instructions = ''

# Индексируем базу знаний
if "github.com" in knowledge_base_link:
  knowledge_base_text = load_text_from_github(knowledge_base_link)
elif "docs.google.com" in knowledge_base_link:
  knowledge_base_text = load_googledoc_by_url(knowledge_base_link)
else:
  knowledge_base_text = load_document_text(knowledge_base_link)
knowledge_base_index = create_search_index(knowledge_base_text, 0, 0, verbose, 0)

history = ChatMessageHistory()
Вопрос_клиента = ''
memory_prompt_template = '''
if the Human introduced themselves (mentioned their name), make sure to include their name in the summary.
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think it is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.

New lines of conversation:
Human: What do you think about climate change? By the way I'm Nisha.
AI: The discussion on climate change often involves diverse perspectives, strategies, and approaches to address the issue.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human shifts the topic and inquires about climate change. AI suggests that this question includes different points of view, strategies and approaches to solving the problem

END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
'''
conv_list=[]
while True:

  Вопрос_клиента = input('\nВопрос клиента: ')
  if Вопрос_клиента.lower() in ['stop','стоп']:
      break
  # записываем вопрос в историю мемори
  history.add_user_message(Вопрос_клиента)
  formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  conv_list.append(f'{formatted_datetime} Клиент: {Вопрос_клиента}\n')
  memory = ConversationSummaryMemory.from_messages(
    llm=lc_OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True,
    prompt_template=memory_prompt_template
    )
  # Извлекаем текст сообщений из history
  user_questions_from_history = [message.content for message in history.messages]
  # Объединяем текст в одну строку
  user_questions_string_from_history = ' '.join(user_questions_from_history)
# если в параметр history подать user_questions_string_from_history, то это история без саммари
# если подать buffer, то это только саммаризация предыдущ диалога

  chunks, output1 = answer_index(system_prompt, Вопрос_клиента, instructions, knowledge_base_index, temperature, verbose, num_fragment, model, hist=memory.buffer)

  # записываем ответ в историю мемори
  history.add_ai_message(output1)
  formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  conv_list.append(f'{formatted_datetime} AI: {output1}\n')
  print("Ответ:\n", output1)

  formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open(f'{formatted_datetime}_dialog.txt', 'w') as file:
  file.writelines(conv_list)

Вопрос клиента: привет, я Вероника. это лазерлов?
Ответ:

 Здравствуйте, Вероника! Да, я представляю компанию LaserLove. Чем могу помочь? У вас есть вопросы о наших услугах или акциях? 🌟
Вопрос клиента: есть у вас лпг?
Ответ:

 Да, у нас есть услуга ЛПГ массажа. Мы проводим процедуры на оборудовании Pascal, которое обеспечивает высокое качество массажа. Хотели бы узнать больше о процедуре или записаться на прием? 🌟
Вопрос клиента: а что это за оборудование?
Ответ:

 Оборудование Pascal для ЛПГ-массажа - это высокотехнологичное оборудование, обеспечивающее эффективный вакуумный массаж. Оно имеет несколько видов манипул, включая вакуумно-роликовые с пассивно вращающимися роликами и подвижный вакуум с активно вращающимися роликами. Это оборудование обеспечивает механическую проработку жировых клеток и стимулирует лимфатическую систему для уменьшения отечности тканей. Готовы познакомить вас с этим оборудованием ближе или записать на процедуру? 🌟
Вопрос клиента: костюм можно принести свой н

In [7]:
print(user_questions_from_history)
print(user_questions_string_from_history)
print(memory.buffer)

['привет', 'Здравствуйте! Как я могу помочь вам сегодня? У вас есть вопросы о наших услугах или продуктах?', 'меня зовут Вероника, интересуюсь ЛПГ', 'Здравствуйте, Вероника! Рады вашему интересу к ЛПГ. У нас в LaserLove предоставляются услуги ЛПГ-массажа для коррекции фигуры и улучшения состояния кожи. Хотели бы узнать больше о наших процедурах или записаться на консультацию? 🌟', 'сколько это стоит?']
привет Здравствуйте! Как я могу помочь вам сегодня? У вас есть вопросы о наших услугах или продуктах? меня зовут Вероника, интересуюсь ЛПГ Здравствуйте, Вероника! Рады вашему интересу к ЛПГ. У нас в LaserLove предоставляются услуги ЛПГ-массажа для коррекции фигуры и улучшения состояния кожи. Хотели бы узнать больше о наших процедурах или записаться на консультацию? 🌟 сколько это стоит?

The human greets the AI and the AI responds with a polite greeting, asking if the human has any questions about their services or products. The AI then explains that LaserLove provides services for laser m